In [2]:
import os
import cv2
import shutil
import random
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\vales\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
def preprocesar_imagen(ruta_origen, ruta_destino, size=(1280, 1280)):
    """
    Lee una imagen, aplica preprocesamiento y la guarda en ruta_destino.
    """
    img = cv2.imread(ruta_origen, cv2.IMREAD_COLOR)
    if img is None:
        print(f"⚠ Error al leer {ruta_origen}")
        return False

    # 1. Aumentar contraste
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    cl = clahe.apply(l)
    lab = cv2.merge((cl, a, b))
    contrastada = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

    # 2. Reducción de ruido
    denoised = cv2.fastNlMeansDenoisingColored(contrastada, None, 10, 10, 7, 21)

    # 3. Resize
    resized = cv2.resize(denoised, size)

    # 4. Guardar
    cv2.imwrite(ruta_destino, resized)
    return True


In [ ]:
def dividir_dataset_con_anotaciones(dataset_original, dataset_dividido,
                                    train_ratio=0.7, val_ratio=0.2, test_ratio=0.1):
    """
    Divide imágenes + anotaciones .txt en train/val/test y preprocesa antes de moverlas.
    """

    # Crear carpetas
    for split in ["train", "val", "test"]:
        os.makedirs(os.path.join(dataset_dividido, split, "images"), exist_ok=True)
        os.makedirs(os.path.join(dataset_dividido, split, "labels"), exist_ok=True)

    # Listar imágenes originales
    imagenes = [f for f in os.listdir(dataset_original)
                if f.lower().endswith((".png", ".jpg", ".jpeg"))]

    random.shuffle(imagenes)

    n = len(imagenes)
    n_train = int(n * train_ratio)
    n_val = int(n * val_ratio)

    train_imgs = imagenes[:n_train]
    val_imgs = imagenes[n_train:n_train + n_val]
    test_imgs = imagenes[n_train + n_val:]

    # Helper interno
    def mover_con_label(lista_imgs, split_name):
        folder_img = os.path.join(dataset_dividido, split_name, "images")
        folder_lbl = os.path.join(dataset_dividido, split_name, "labels")

        for img in lista_imgs:
            ruta_img = os.path.join(dataset_original, img)
            ruta_lbl = os.path.splitext(ruta_img)[0] + ".txt"

            # Validar que haya anotación
            if not os.path.exists(ruta_lbl):
                print(f"⚠ La imagen {img} NO tiene anotación TXT. Saltando.")
                continue

            # Preprocesar ANTES de mover
            destino_img = os.path.join(folder_img, img)
            preprocesar_imagen(ruta_img, destino_img)

            # Copiar anotación
            destino_lbl = os.path.join(folder_lbl, os.path.splitext(img)[0] + ".txt")
            shutil.copy(ruta_lbl, destino_lbl)

    mover_con_label(train_imgs, "train")
    mover_con_label(val_imgs, "val")
    mover_con_label(test_imgs, "test")

    print("✅ División de dataset completada (con anotaciones).")


In [ ]:
def entrenar_yolo(config_yaml, epochs=50, imgsz=1280):
    """
    Entrena YOLOv8n utilizando la configuración YAML del dataset.
    """
    print("🚀 Cargando modelo YOLOv8n...")
    model = YOLO("yolov8n.pt")

    print("🟢 Iniciando entrenamiento...")
    results = model.train(
        data=config_yaml,
        epochs=epochs,
        imgsz=imgsz,
        batch=4,
        device="cuda" if torch.cuda.is_available() else "cpu"
    )

    print("🏁 Entrenamiento finalizado.")
    return model, results


In [ ]:
def validar_modelo(modelo, ruta_yaml):
    """
    Ejecuta validación sobre el dataset de validación.
    """
    print("📊 Ejecutando validación...")
    metrics = modelo.val(data=ruta_yaml)
    print("📈 Métricas:", metrics)
    return metrics


# Ejecucion de Pipline

In [ ]:
#Configuracion de rutas

dataset_original = "../dataset_imagenes"
dataset_dividido = "./dataset_dividido"
ruta_yaml = "./etiquetas_yolo.yaml"


In [ ]:
# Dividir Dataset
dividir_dataset_con_anotaciones(dataset_original, dataset_dividido)

In [ ]:
# Entrenar modelo
modelo, resultados = entrenar_yolo(ruta_yaml, epochs=50)

In [ ]:
# Validar modelo
validar_modelo(modelo, ruta_yaml)